# A dead simple markdown transformer for code cells

Previous implementations used python_line_transforms to transform the code.  With that approach a lot of effort was required to make sure that any other transformers were executed in the proper order.

In this experiment, the markdown transformer is baked directly in the `input_transform_manager`.

    %reload_ext deathbeds.__Markdown_code_cells

In [1]:
    from CommonMark import Parser
    from CommonMark.render.renderer import Renderer
    from textwrap import indent, dedent
    from IPython.display import display, Markdown
    from IPython.core.inputtransformer import InputTransformer
    from importnb import Interactive
    from collections import UserList
    from abc import abstractmethod, ABCMeta

* Create a custom CommonMark renderer for code only.  It maintains the lines and positions of the original source to give better error messages.

* The commonmark renderer only catches code cells

    > There is an interesting discussion to be had about the role of inline cells.

In [2]:
    class CodeRenderer(Renderer):
        def code_block(self, node, entering):
            while len(self.buf.splitlines()) <  node.sourcepos[0][0]-1:
                self.out("\n")
            self.out(indent(node.literal, ' '*(
                node.sourcepos[0][1])))
            
        def __call__(self, str):
            return self.render(Parser().parse(str))

    def render(str): 
        return dedent(CodeRenderer().render(Parser().parse(str)))

In [3]:
    class CallableTransformer(InputTransformer, UserList, metaclass=ABCMeta):
        """Define a __call__ method define a callable transformer."""
        
        push = UserList.append

        def reset(self, str = ""):
            while self.data: str += self.data.pop(0) + '\n'
            return self(str)
        
        @abstractmethod
        def __call__(self, str): raise NotImplemented()
            
        

* The `Literate` Callable transformer will concatenate block code in a contigious code block.

In [4]:
    class Literate(CallableTransformer):
        def __call__(self, str):
            if str.split('\n', 1)[0].strip(): display(Markdown(str))
            return render(str)

* Append a physical line transform to the input transform manager.  This is only available in the interactive context.

In [5]:
    def load_ipython_extension(ip=None):
        ip = ip or get_ipython()
        if not any(isinstance(object, Literate) 
                   for object in ip.input_transformer_manager.physical_line_transforms):
            ip.input_transformer_manager.physical_line_transforms.insert(0, Literate())
            
    def unload_ipython_extension(ip=None):
        ip.input_transformer_manager.physical_line_transforms = list(
            object for object in ip.input_transformer_manager.physical_line_transforms
            if not isinstance(object, Literate)
        )

## Importing Markdown style files

If documents can be composed using Markdown forward syntax then they should be importable.  The extension `.md.ipynb` will identify markdown forward notebooks.

        if __name__ == '__main__':
            with MarkdownImporter(shell=False):
                import Untitled1 as nb

In [6]:
    class MarkdownImporter(Interactive):
        extensions = '.md.ipynb',
        def format(self, str): return super().format(render(str))

* Test the Markdown Splitter

In [7]:
    if __name__ == '__main__':
        with MarkdownImporter():
            %load_ext deathbeds.__String_Node_Transformer
            import deathbeds.__Markdown_code_cells
            %load_ext deathbeds.__Markdown_code_cells

## Thoughts ?

> * The Markdown transformer must be a physical_line_transform because they come first; as long as the class is an inputtransformer class things will work.
> * This transformer should work with a template transformer.



In [9]:
    
    if __name__ == '__main__': import disqus